In [ ]:
import cudf
from cuml.tsa.arima import ARIMA

import matplotlib.pyplot as plt
import warnings
from warnings import catch_warnings
from warnings import filterwarnings
import datetime
from ast import literal_eval
import numpy as np

filterwarnings('ignore')

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

def cal_duration(time1, time2):
    td = time2 - time1
    return td.total_seconds()

if __name__ == '__main__':
    # load dataset
    df = cudf.read_csv("eurusd_h1_2016_1_2021_10.csv", usecols=[4])
    df = df.values
    # df = df.flatten()

    #Calculate the natural logarithm of the data so that the first difference 
    #gets the data percentage growth.
    df = np.log(df)
    print(df[0:10])
    # for i in range(2, 50):
    i = 1
    n_test = 24
    # all_data = df[i * n_test:( i * n_test) + 3000]
    # all_data = df[0:100]
    data = all_data[:-n_test] # For test of prediction last predict_step steps
    
    plt.plot(data)
    plt.show()
    # order = (1, 1, 1)
#     s_order = (1, 1, 1, 30)
#     time1 = datetime.datetime.now()

#     # Build Model to test the result with the data_test
#     model = ARIMA(data, order=order, seasonal_order=s_order, fit_intercept=False)
#     model_fit = model.fit()
#     prediction = model_fit.forecast(n_test)

#     # data=np.exp(data)
#     actual=np.exp(all_data[-n_test:len(all_data)])
#     prediction= np.exp(prediction)
#     time2 = datetime.datetime.now()
#     print('------------duration-----------')
#     print(cal_duration(time1, time2))
#     print("===========prediction==========")
#     print(prediction)
#     print("===========actual==========")
#     print(actual)
#     #Plot prediction n steps ahead
#     plt.plot(actual, label='actual')
#     plt.plot(prediction, label='prediction')
#     plt.title('Prediction vs Actual')
#     plt.legend(loc='upper left', fontsize=8)
#     plt.savefig('test.png')
#     plt.close()
#     # plt.show()